### This script is based on simple features derived from text variables. 

In [167]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB

### load data

In [168]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [169]:
print ('The train data has {} rows and {} columns'.format(train.shape[0],train.shape[1]))
print ('The test data has {} rows and {} columns'.format(test.shape[0],test.shape[1]))

The train data has 108129 rows and 14 columns
The test data has 63465 rows and 12 columns


### convert time to unix format

In [170]:
import time

unix_cols = ['deadline','state_changed_at','launched_at','created_at']

for x in unix_cols:
    train[x] = train[x].apply(lambda k: time.ctime(k))
    test[x] = test[x].apply(lambda k: time.ctime(k))

### create simple features

In [171]:
cols_to_use = ['name','desc']
len_feats = ['name_len','desc_len']
count_feats = ['name_count','desc_count']

for i in np.arange(2):
    train[len_feats[i]] = train[cols_to_use[i]].apply(str).apply(len)
    train[count_feats[i]] = train[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))

In [172]:
train['keywords_len'] = train['keywords'].apply(str).apply(len)
train['keywords_count'] = train['keywords'].apply(str).apply(lambda x: len(x.split('-')))

In [173]:
for i in np.arange(2):
    test[len_feats[i]] = test[cols_to_use[i]].apply(str).apply(len)
    test[count_feats[i]] = test[cols_to_use[i]].apply(str).apply(lambda x: len(x.split(' ')))
    
test['keywords_len'] = test['keywords'].apply(str).apply(len)
test['keywords_count'] = test['keywords'].apply(str).apply(lambda x: len(x.split('-')))

### encoding features

In [174]:
from sklearn.preprocessing import LabelEncoder

feat = ['disable_communication','country']
for x in feat:
    le = LabelEncoder()
    le.fit(list(train[x].values) + list(test[x].values))
    train[x] = le.transform(list(train[x]))
    test[x] = le.transform(list(test[x].values))
    

### model training

In [175]:
cols_to_use = ['name_len'
                 ,'desc_len'
                 ,'keywords_len'
                 ,'name_count'
                 ,'desc_count'
                 ,'keywords_count']

target = train['final_status']

# data for modeling
k_train = train[cols_to_use]
k_test = test[cols_to_use]

### naive bayes

In [176]:
gnb = GaussianNB()
nvb_pred = gnb.fit(k_train, target).predict(k_test)

### write the file

In [177]:
nBsub = pd.DataFrame({'project_id':test['project_id'],'final_status':nvb_pred})
nBsub = nBsub[['project_id','final_status']]
nBsub.to_csv("nBstarter.csv",index = False) #0.6526